In [1]:
"""
该项目运行环境：Windows 10
python =3.6
相关算法包环境
tensorflow = 2.0.0
numpy =1.18.3
"""


from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import numpy as np
import datetime
print(tf.__version__)
print(np.__version__)

2.0.0
1.17.1


In [2]:
tf.test.is_gpu_available()

True

In [3]:
mnist = np.load("mnist.npz")
x_train, y_train, x_test, y_test = mnist['x_train'],mnist['y_train'],mnist['x_test'],mnist['y_test']

x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]


train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)


In [4]:

class MyModel(Model):
    def __init__(self,**kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')
    @tf.function
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# model = tf.keras.models.Sequential([
#     tf.keras.layers.Flatten(input_shape=(28, 28,1)),
#     tf.keras.layers.Dense(32, activation='relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(10, activation='softmax')
# ])

In [5]:

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


# @tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)


# @tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)



In [6]:
model = MyModel()

In [7]:
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
import os
logdir = os.path.join("logs/"+stamp)
 

summary_writer = tf.summary.create_file_writer(logdir) 

EPOCHS = 1

for epoch in range(EPOCHS):
    for (x_train, y_train) in train_ds:

            train_step(x_train, y_train)
            
        
    with summary_writer.as_default():                               # 希望使用的记录器
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)  # 还可以添加其他自定义的变量

#     for (x_test, y_test) in test_ds:
#         test_step(x_test, y_test)


#     template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
#     print(template.format(epoch + 1,
#                           train_loss.result(),
#                           train_accuracy.result() * 100,
#                           test_loss.result(),
#                           test_accuracy.result() * 100))

    # Reset metrics every epoch
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()
    
with summary_writer.as_default():
    tf.summary.trace_on(graph=True, profiler=False)  # 开启Trace，可以记录图结构和profile信息
    
    tf.summary.trace_export(name="model_trace", step=3, profiler_outdir=None)    # 保存Trace信息到文件




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [10]:
!rm -rf ./logs/*

rm: 无法删除'./logs/20200524-192931': 目录非空


In [13]:
summary_writer = tf.summary.create_file_writer('./tensorboard') 
log_dir = "graph"



NameError: name 'summary_writer' is not defined

In [12]:
summary_writer = tf.summary.create_file_writer('./tensorboard') 

# train_summary_writer = tf.summary.create_file_writer(train_log_dir)
# test_summary_writer = tf.summary.create_file_writer(test_log_dir)

EPOCHS = 5

for epoch in range(EPOCHS):
    for (x_train, y_train) in train_ds:
        train_step(x_train, y_train)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    for (x_test, y_test) in test_ds:
        test_step(x_test, y_test)
#     with test_summary_writer.as_default():
#         tf.summary.scalar('loss', test_loss.result(), step=epoch)
#         tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

    tf.summary.trace_on(graph=True, profiler=True)  # 开启Trace，可以记录图结构和profile信息
    # 进行训练
    with summary_writer.as_default():
        tf.summary.trace_export(name="model_trace", step=0, profiler_outdir=log_dir)    # 保存Trace信息到文件


        
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))

    # Reset metrics every epoch
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

W0521 22:35:09.359450 15260 base_layer.py:1772] Layer my_model is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

W0521 22:35:09.969441 15260 deprecation.py:323] From d:\miniconda3\lib\site-packages\tensorflow_core\python\ops\math_grad.py:1394: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
U

Epoch 1, Loss: 0.1391400545835495, Accuracy: 95.88500213623047, Test Loss: 0.06750461459159851, Test Accuracy: 97.81999969482422
Epoch 2, Loss: 0.04322267323732376, Accuracy: 98.66166687011719, Test Loss: 0.05358003079891205, Test Accuracy: 98.3499984741211
Epoch 3, Loss: 0.02376703917980194, Accuracy: 99.2683334350586, Test Loss: 0.051972296088933945, Test Accuracy: 98.32999420166016
Epoch 4, Loss: 0.01276030670851469, Accuracy: 99.56832885742188, Test Loss: 0.062477342784404755, Test Accuracy: 98.37999725341797
Epoch 5, Loss: 0.009908459149301052, Accuracy: 99.70333862304688, Test Loss: 0.06805026531219482, Test Accuracy: 98.43000030517578
